In [1]:
from sklearn.datasets import load_digits
digits=load_digits()
dir(digits)

['DESCR', 'data', 'feature_names', 'frame', 'images', 'target', 'target_names']

In [2]:
import pandas as pd
df=pd.DataFrame(digits.data,columns=digits.feature_names)
df['target']=digits.target
df.head()

,pixel_0_0,pixel_0_1,pixel_0_2,pixel_0_3,pixel_0_4,pixel_0_5,pixel_0_6,pixel_0_7,pixel_1_0,pixel_1_1,...,pixel_6_7,pixel_7_0,pixel_7_1,pixel_7_2,pixel_7_3,pixel_7_4,pixel_7_5,pixel_7_6,pixel_7_7,target
0,0.0,0.0,5.0,13.0,9.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,6.0,13.0,10.0,0.0,0.0,0.0,0
1,0.0,0.0,0.0,12.0,13.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,11.0,16.0,10.0,0.0,0.0,1
2,0.0,0.0,0.0,4.0,15.0,12.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,3.0,11.0,16.0,9.0,0.0,2
3,0.0,0.0,7.0,15.0,13.0,1.0,0.0,0.0,0.0,8.0,...,0.0,0.0,0.0,7.0,13.0,13.0,9.0,0.0,0.0,3
4,0.0,0.0,0.0,1.0,11.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,2.0,16.0,4.0,0.0,0.0,4


In [3]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier

In [6]:
model_params={
    'svm':{
        'model':SVC(gamma='auto'),
        'params':{
             'C': [0.1, 1, 10, 100],            # Regularization
        'kernel': ['linear', 'rbf', 'poly'],
        'degree': [2, 3, 4],               # Only used for 'poly'
        'gamma': ['scale', 'auto'],        # Kernel coefficient
        'probability': [True]              # Enable probability estimates
        }
        },
    'random_forest':{
        'model':RandomForestClassifier(),
        'params':{
            'n_estimators': [10, 50, 100],
        'max_depth': [None, 5, 10],
        'min_samples_split': [2, 5],
        'min_samples_leaf': [1, 2],
        'max_features': ['auto', 'sqrt', 'log2'],
        'bootstrap': [True, False]
        }
    },
    'logistic_regression':{
        'model':LogisticRegression(solver='liblinear',multi_class='auto'),
        'params':{
            'C': [0.01, 0.1, 1, 10, 100],
        'penalty': ['l1', 'l2'],           # Type of regularization
        'solver': ['liblinear'],           # Supports both l1 and l2
        'fit_intercept': [True, False]
        }
    },
    'gaussian_nb':{
        'model':GaussianNB(),
        'params':{
            'var_smoothing': [1e-9, 1e-8, 1e-7]  # Helps avoid zero division, useful on noisy data
        }
    },
    'multinomial_nb':{
        'model':MultinomialNB(),
        'params':{
            'alpha': [0.1, 0.5, 1.0],          # Smoothing parameter
            'fit_prior': [True, False]         # Learn class priors or assume uniform
        }
    },
    'decision_tree':{
        'model':DecisionTreeClassifier(),
        'params':{
            'criterion': ['gini', 'entropy'],
            'max_depth': [None, 5, 10],
            'min_samples_split': [2, 5],
            'min_samples_leaf': [1, 2],
            'max_features': [None, 'sqrt', 'log2']
        }
    },

}

In [7]:
scores = []

for model_name, mp in model_params.items():
    clf =  GridSearchCV(mp['model'], mp['params'], cv=5, return_train_score=False)
    clf.fit(digits.data, digits.target)
    scores.append({
        'model': model_name,
        'best_score': clf.best_score_,
        'best_params': clf.best_params_
    })

resultdf = pd.DataFrame(scores,columns=['model','best_score','best_params'])
resultdf

/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_validation.py:528: FitFailedWarning: 
360 fits failed out of a total of 1080.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
360 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.11/dist-packages/sklearn/base.py", line 1382, in wrapper
    estimator._validate_params()
  File "/usr/local/lib/python3.11/dist-packages/sklearn/base.py", line 436, in _validate_params
    validate_parameter_constraints(
  File "/usr/local/lib/python3.11/dist-packages/sklearn/uti

,model,best_score,best_params
0,svm,0.973850,"{'C': 10, 'degree': 2, 'gamma': 'scale', 'kern..."
1,random_forest,0.945484,"{'bootstrap': False, 'max_depth': None, 'max_f..."
2,logistic_regression,0.930458,"{'C': 0.1, 'fit_intercept': False, 'penalty': ..."
3,gaussian_nb,0.832518,{'var_smoothing': 1e-07}
4,multinomial_nb,0.870907,"{'alpha': 0.1, 'fit_prior': True}"
5,decision_tree,0.814729,"{'criterion': 'entropy', 'max_depth': 10, 'max..."
